# Text Clustering

Text clustering is a major task in NLP and in other areas, and is one of the most common task for unspervised learning

In this notebook, I am going to apply **unsupervised learning** techniques to group similar news articles from the BBC News dataset. Using different **text vectorization methods**, I am going to explore how well various clustering algorithms can categorize the news articles into meaningful groups.

 What You’ll Learn by Doing This Project
How to apply clustering algorithms to text data.
The impact of different text representations on clustering quality.
How to evaluate clustering performance (even without predefined labels).
The practical challenges of unsupervised learning in NLP.

## Steps Overview
1. **Load & Prepare Data**  
   - Load preprocessed text and vectorized representations (TF-IDF, Word2Vec, SBERT).

2. **Apply Clustering Algorithms**  
   - Use **K-Means**, **Hierarchical Clustering**, and **DBSCAN** to identify clusters.

3. **Evaluate Clustering Performance**  
   - Compute **Silhouette Score**, **Davies-Bouldin Index**, and **Adjusted Rand Index (ARI)** (if labels are available).

4. **Visualize Clusters**  
   - Reduce dimensionality using **PCA or t-SNE**.
   - Plot cluster distributions to analyze how well-separated they are.

5. **Compare Embedding Methods**  
   - Evaluate clustering results for **TF-IDF, Word2Vec, and SBERT**.
   - Discuss which embedding method is most effective for clustering news articles.

## 🛠 Tech Stack
- **Python** (pandas, numpy, matplotlib, seaborn)
- **scikit-learn** (K-Means, evaluation metrics)
- **gensim** (Word2Vec)
- **sentence-transformers** (SBERT embeddings)
- **hdbscan** (Density-based clustering)

## 🎯 Expected Outcome
- A clear comparison of **how different embeddings affect clustering**.
- Identification of **the best approach for grouping news articles** without predefined labels.
- Insights into **how well clusters align with actual news categories**.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer

### 1. **Load & Prepare Data**

In [ ]:
df = pd.read_csv("bbc-text.csv")

In [ ]:
nlp = spacy.load("en_core_web_sm")  # load the english NLP model

def preprocess_texts(texts):
    processed_texts = []
    for doc in nlp.pipe(texts, disable=["ner", "parser"]):  
        tokens = [token.lemma_ for token in doc if not token.is_punct and not token.is_stop]
        processed_texts.append(" ".join(tokens))
    return processed_texts

df["processed_text"] = preprocess_texts(df["text"])

In [ ]:
# TF-IDF 

tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # stick with the 5000 most common words
tfidf_matrix = tfidf_vectorizer.fit_transform(df["processed_text"]).toarray()

In [ ]:
# Word2Vec 

texts = df["processed_text"].tolist()
w2v_model = Word2Vec(sentences=[text.split() for text in texts], vector_size=300, window=5,  min_count=2, workers=4)
word_vectors = np.array([np.mean([w2v_model.wv[word] for word in text.split() if word in w2v_model.wv], axis=0) for text in texts])

In [ ]:
# BERT

sbert_model = SentenceTransformer('all-MiniLM-L6-v2') 
df['sbert_embedding'] = df['processed_text'].apply(lambda x: sbert_model.encode(x))
sbert_embeddings = np.stack(df['sbert_embedding'].values)


### 2. **Apply Clustering Algorithms**

#### K-Means clustering

K-means is an unsupervised clustering algorithm that partitions data into k clusters by iteratively updating cluster centroids to minimize intra-cluster variance (sum of squared distances). It is simple, easy to implement, and efficient even on large datasets, making it widely used. On the flip side, K-means performs best when clusters are roughly spherical and of similar size, which is not always the case. Additionally, the number of clusters (k) must be specified beforehand, which can be a challenging problem all by itself.

#### How to select the number of clusters to initialize K-Means algorithm?

Before applying the K-Means algorithm, I will use both the **elbow method** and the **silhouette score** to determine the optimal number of clusters. Since they have different strengths, using them together provides a more well-rounded analysis, especially in cases where the optimal k is not clear.

#### Elbow method

It measures how compact clusters are by evaluating within-cluster variance (or inertia). This technique helps determine the optimal number of clusters by identifying the point where adding more clusters no longer significantly reduces inertia (this is a key point to make since adding more clusters always reduces the distance between points and their assigned centroids). In this regard detecting the elbow isn't always clear. 

This method is particularly useful for datasets with well-defined clusters, as it focuses solely on how tightly data points are grouped rather than how well-separated the clusters are.

In [ ]:
import elbow

#### Silhouette score

It takes into account both cohesion (similarity inside the cluster) and separation (differences between clusters). A higher score means better-defined clusters. For a suscint and clear explanation see [here](https://www.youtube.com/watch?v=a2Kg2_l3L8M) 

In [ ]:
import silhouette

Let's apply the elbow method to the vectors generated with the TF-IDF method:

In [ ]:
elbow.elbow_method(tfidf_matrix, max_k=10)

**How to interpret this plot?**

This plot shows the inertia as a function of the number of clusters. The fact that the inertia is a monotonic decreasing function is expected.

Here we have to look for the "elbow" point: this is where the rate of decrease in inertia slows down significantly. In this plot however, there is no clear elbow. The decrease appears quite linear (with the same slope value that is) across all the domain of k. But the largest drop happens between $k = 1$ and $k = 3$ and after that the reduction appears more gradual. If forced to choose based on this plot alone, $k=3$ might be reasonable.

Since the elbow method appears to be non-conclusive, let's check now the **silhouette score**

In [ ]:
silhouette.silhouette_method(tfidf_matrix, max_k=15)

**How to interpret this plot?**

As mentioned before, the silhouette score measures clustering quality ( higher values indicate better defined and well separated clusters)

The silhouette score at $k = 3$ is low compared to higher k values. This suggest that increasing k improves clustering quality.

At $k = 8$ the silhouette score reaches a local maximum, which might indicate distinct and well-separated clusters.

The drop after this value may suggests this well defined clusters are being split into smaller, less meaningful ones (this is known as *over-segmentation*)

The increase around $k = 14$ could mean that there is an *underlying substructure*, and subsequent partition of the data helps to capture this. However, too high k risks overfitting by artificially creating more clusters than needed.


**Why is the silhouette score more reliable in this case?**

The elbow method only considers inertia, which measures how compact clusters are. However, compact clusters do not always mean good separation. The silhouette score takes into acount both intra and inter cluster distance.

Now let's apply this methods to the vectors generated with Word2Vec:

In [ ]:
# Apply elbow method

elbow.elbow_method(word_vectors)

Here, $k = 4$ or $k = 5$ are good candidates: inertia decreases much more slowly for $k > 5$

In [ ]:
# Calculate silhouette score 

silhouette.silhouette_method(word_vectors)

Silhouette score is the greatest for $k = 4$. In this case, the k's obtained with both methods coincide!

Finally, apply this methods to the vectors generated with SBERT:

In [ ]:
# Apply elbow method

elbow.elbow_method(sbert_embeddings)

Here, $k = 4$, $k = 5$ or even $k = 7$ are possible candidates. 

In [ ]:
# Calculate silhouette score 

silhouette.silhouette_method(sbert_embeddings)

Applying a similar criteria as the one in the first case, I will select $k = 4$ as the best candidate.

Then the number of clusters for initializing K-Means for the three arrays of vectors are:

$$
\begin{array}{|c|c|c|c|}
\hline
\text{\# clusters} & \text{TF-IDF} & \text{Word2Vec} & \text{SBERT} \\
\hline
k & 8 & 4 & 4 \\
\hline
\end{array}
$$


Now it's time to apply the K-Means algorithm.

In [ ]:
import kmeans

In [ ]:
# Apply K-Means clustering to the generated vectors

tfidf_labels, tfidf_inertia = kmeans.kmeans_clustering(tfidf_matrix, k=8)
word_labels, word_inertia = kmeans.kmeans_clustering(word_vectors, k=4)
sbert_labels, sbert_inertia = kmeans.kmeans_clustering(sbert_embeddings, k=4)

### Hierarchical Clustering

Hierarchical clustering builds a *hierarchy of clusters* that can be cut at different levels to obtain different groupings. It works in two main ways, top-down and bottom-up (agglomerative) approaches, with the bottom-up method being the most common.

The result of this process is a tree-like graph called a **dendrogram**.  This kind of plots give a clear visualization of data relationships, for instance, the height at which two clusters merge represents their distance. By cutting the dendrogram at a specific height, you can determine the final clusters.

One strength of hierarchical clustering, compared to methods like k-means, is that it doesn’t require predefining the number of clusters. On the flip side it becomes computationally expensive for large datasets and is sensitive to noise and outliers.
